In [26]:
import pandas as pd

In [27]:
# csv 파일 불러오기
df = pd.read_csv("서울시 관광특구 주요공원 리스트.csv")

# 컬럼 이름 변경
df = df.rename(columns={"구": "gu", "공원명": "park_name", "네이버맵 기준 블로그 리뷰 수": "navermap_reviews_count", "카카오맵 기준 블로그 리뷰 수": "kakaomap_reviews_count", "리뷰 수 총합": "total_reviews"})

# 관광 특구만 뽑아내는 작업 필요
df = df[df["gu"].isin(["중구", "용산구", "종로구", "송파구", "강남구", "마포구"])]

# 용산구 별도 저장
yongsan_df = df[df["gu"] == "용산구"]

# 용산구 외 데이터
other_df = df[df["gu"] != "용산구"]

In [28]:
df.head()

,gu,park_name,navermap_reviews_count,kakaomap_reviews_count,total_reviews
0,중구,서울로 7017,3282.0,192.0,3474
1,중구,남산공원,1216.0,124.0,1340
2,중구,서소문역사공원,396.0,45.0,441
3,중구,손기정체육공원,254.0,24.0,278
4,중구,훈련원근린공원(훈련원공원),104.0,10.0,114


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38 entries, 0 to 37
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   gu                      38 non-null     object 
 1   park_name               38 non-null     object 
 2   navermap_reviews_count  38 non-null     float64
 3   kakaomap_reviews_count  38 non-null     float64
 4   total_reviews           38 non-null     int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 1.8+ KB


In [30]:
# 구 별로 total_review를 기준으로 정렬
df_sorted = df.sort_values(["gu", "total_reviews"], ascending=[True, False])

In [31]:
df.head()

,gu,park_name,navermap_reviews_count,kakaomap_reviews_count,total_reviews
0,중구,서울로 7017,3282.0,192.0,3474
1,중구,남산공원,1216.0,124.0,1340
2,중구,서소문역사공원,396.0,45.0,441
3,중구,손기정체육공원,254.0,24.0,278
4,중구,훈련원근린공원(훈련원공원),104.0,10.0,114


In [32]:
# 구별 Q3 이상 공원만 선택할 수 있게 하는 함수
def select_q3(group):
    q3 = group["total_reviews"].quantile(0.75)
    return group[group["total_reviews"] >= q3]

In [33]:
# 전체 기준: 상위 15% 이상인 공원 추출
q85 = df["total_reviews"].quantile(0.85)
highly_reviewed_df = df[df["total_reviews"] >= q85]

In [34]:
highly_reviewed_df

,gu,park_name,navermap_reviews_count,kakaomap_reviews_count,total_reviews
6,용산구,용산가족공원,3745.0,401.0,4146
8,종로구,인왕산도시자연공원(인왕산),8249.0,331.0,8580
9,종로구,낙산공원,6192.0,296.0,6488
20,송파구,올림픽공원,22371.0,1546.0,23917
21,송파구,송파나루근린공원(석촌호수),20825.0,1146.0,21971
33,마포구,경의선숲길,4935.0,34.0,4969


In [35]:
# 구별 Q3 기준 + 전체 상위 15% 병합
q3_selection = other_df.groupby("gu", group_keys=False).apply(select_q3)

# 중복 제거 + 병합
final_df = pd.concat([q3_selection, yongsan_df, highly_reviewed_df], ignore_index=True).drop_duplicates()

/var/folders/2l/hl66_jcn0_9d8l_bdq3bjxwc0000gn/T/ipykernel_96755/2751591826.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  q3_selection = other_df.groupby("gu", group_keys=False).apply(select_q3)


In [ ]:
# 저장
final_df.to_csv("selected_parks.csv", index=False)

# 확인
print(f"최종 선택된 공원 수: {len(final_df)}개")
final_df

최종 선택된 공원 수: 13개


,gu,park_name,navermap_reviews_count,kakaomap_reviews_count,total_reviews
0,강남구,도산근린공원,2081.0,26.0,2107
1,강남구,율현공원,222.0,30.0,252
2,마포구,경의선숲길,4935.0,34.0,4969
3,마포구,문화비축기지,2553.0,196.0,2749
4,송파구,올림픽공원,22371.0,1546.0,23917
5,송파구,송파나루근린공원(석촌호수),20825.0,1146.0,21971
6,종로구,인왕산도시자연공원(인왕산),8249.0,331.0,8580
7,종로구,낙산공원,6192.0,296.0,6488
8,종로구,북한산국립공원(북한산),3993.0,126.0,4119
9,중구,서울로 7017,3282.0,192.0,3474
